In [1]:
##### DNN module

from keras.utils import to_categorical
from keras import backend as K
from keras.models import Model
from keras.layers import Flatten, Dense, Dropout
from keras.applications import ResNet50
from keras.optimizers import Adagrad
#from keras.preprocessing import image

from functools import partial
from collections import defaultdict

Using TensorFlow backend.


In [2]:
##### Common moldule

import numpy as np
import os
import glob
import matplotlib.pyplot as plt
from PIL import Image
from scipy.interpolate import RegularGridInterpolator

In [3]:
##### Personal function

def mx_preview2(array, min=0, max=2048, plotsize=6):
    plt.figure(figsize=(plotsize, plotsize))
    plt.imshow(array, aspect = 'equal', interpolation='nearest')
    plt.title('(x, y) = {}, Int limit = {}'.format(np.shape(array), [min, max]),
              fontsize = 16, y=1.05)
    plt.xlabel('X', fontsize = 16)
    plt.ylabel('Y', fontsize = 16)
    plt.clim(min,max)
    plt.colorbar(fraction=0.0451, pad=0.05)
    return plt.show()

def resampleRGI3d(input_mx, resize_to, dtype='float32'):
    # Ref: 10 3-A-7
    # Ver. 2.1
    # input_mx : numpy array, the original target matrix
    # resize_to: list or tuple with 3 int inside
    a, b, c = np.shape(input_mx)
    p, q, r = resize_to
    z_grid = np.linspace(0, p - 1, a)
    y_grid = np.linspace(0, q - 1, b)
    x_grid = np.linspace(0, r - 1, c)
    RGI = RegularGridInterpolator((z_grid, y_grid, x_grid), input_mx)
    z_grid_t2 = np.arange(p)
    y_grid_t2 = np.arange(q)
    x_grid_t2 = np.arange(r)
    meshgrid_para = np.meshgrid(z_grid_t2, y_grid_t2, x_grid_t2)
    RGI_mesh_mx = RGI((meshgrid_para[0], meshgrid_para[1], meshgrid_para[2]))
    RGI_mx = np.transpose(RGI_mesh_mx, axes=[1, 0, 2]).astype(dtype)
    return RGI_mx

In [4]:
##### Path and Read

path_db       = 'database/image_data'
path_train    = os.path.join(path_db, 'train')
path_test     = os.path.join(path_db, 'test')
sub_folder_li = list(map(os.path.basename, glob.glob(os.path.join(path_train, '*'))))

flower_mapping = {sub_folder_li[i]: i for i in range(5)}

print(flower_mapping)

{'daisy': 0, 'dandelion': 1, 'rose': 2, 'sunflower': 3, 'tulip': 4}


In [5]:
##### Load Training file
#pic_store_dict = defaultdict(partial(np.ndarray, 0))
total_dict = {}
for i in range(len(sub_folder_li)):
    path_train_0 = glob.glob(os.path.join(path_train, list(flower_mapping.keys())[i], '*'))
    pic_store_dict = None
    pic_store_dict = {}
    for j in range(len(path_train_0)):
        serial = os.path.basename(path_train_0[j])[:-4]
        temp_pic = np.asarray(Image.open(path_train_0[j]))
        temp_shape = np.shape(temp_pic)
        if temp_shape[0] > 640:
            temp_pic = temp_pic[:640]
            temp_shape = np.shape(temp_pic)
        if temp_shape[1] > 640:
            temp_pic = temp_pic[:, :640]
            temp_shape = np.shape(temp_pic)
        if temp_shape[0] > 100:
            temp_pic = resampleRGI3d(temp_pic, (240, int(temp_shape[1]*240/temp_shape[0]), 3))
            temp_shape = np.shape(temp_pic)
        if temp_shape[1] > 100:
            temp_pic = resampleRGI3d(temp_pic, (int(temp_shape[0]*240/temp_shape[1]), 240, 3))
            temp_shape = np.shape(temp_pic)
        #if (temp_shape[0] <= 640) and (temp_shape[1] <= 640):
        temp_pic = np.pad(temp_pic, ((0, 100 - temp_shape[0]), (0, 100 - temp_shape[1]), (0, 0)), 'constant', constant_values=255)
        pic_store_dict[serial] = temp_pic
    stack_a = np.stack([pic_store_dict[x] for x in list(pic_store_dict.keys())], axis=0)
    total_dict[sub_folder_li[i]] = stack_a
#print(np.shape(merged_mx))

In [6]:
temp_shape[1]

240

In [7]:
print(np.shape(total_dict['daisy']))
print(np.shape(total_dict['dandelion']))
print(np.shape(total_dict['rose']))
print(np.shape(total_dict['sunflower']))
print(np.shape(total_dict['tulip']))

(500, 240, 240, 3)
(687, 240, 240, 3)
(515, 240, 240, 3)
(488, 240, 240, 3)
(633, 240, 240, 3)


In [8]:
merged_mx = np.concatenate((total_dict['daisy'],
                            total_dict['dandelion'],
                            total_dict['rose'],
                            total_dict['sunflower'],
                            total_dict['tulip']), axis = 0)
print(np.shape(merged_mx))

(2823, 240, 240, 3)


In [9]:
##### Generate Labels

for i in range(len(list(flower_mapping.keys()))):
    if i == 0:
        label_li = [i]* len(glob.glob(os.path.join(path_train, list(flower_mapping.keys())[i], '*')))
    else:
        label_li += [i]* len(glob.glob(os.path.join(path_train, list(flower_mapping.keys())[i], '*')))
label_mx = np.array(label_li)
print(np.shape(label_mx))

(2823,)


In [10]:
##### Save training matrix and label matrix to file
np.save('database/combined_train_mx_240.npy', merged_mx.astype('uint8'))
np.save('database/combined_label_mx_240.npy', label_mx.astype('uint8'))

In [ ]:
#####
#####
#####